# Global SDG Data Series Catalogue
This notebook illustrates a series of steps necessary to create a catalog of the data series included in the Global SDG Indicator Framework

## Initial settings

In [3]:
import pandas as pd
import os 

dir_path = os.path.dirname(os.path.realpath('__file__'))
print(dir_path)

global_indicator_dir = r'../Global Indicator Framework/'


print('global indicator framework dir: ' + global_indicator_dir)


output_dir = r'../SDG-Matrix/Output/'
print('sdg matix outputs dir: ' + output_dir)
print('Initialization completed')



/home/jovyan/PCBS/notebooks
global indicator framework dir: ../Global Indicator Framework/
sdg matix outputs dir: ../SDG-Matrix/Output/
Initialization completed


## Global indicator framework

In [ ]:
indicator_list = pd.read_excel(global_indicator_dir + 'IndicatorList_20190323.xlsx')
indicator_list.head(10)

## Tier classification

In [ ]:
tier_classification = pd.read_excel(global_indicator_dir + 'TierCllassification_20190213.xlsx')
tier_classification.head(10)

## Global data series

In [ ]:
series = pd.read_json('https://unstats.un.org/SDGAPI/v1/sdg/Series/List?allreleases=false')
series.head(10)

In [ ]:
global_series = []

for index, row in series.iterrows():
    
    series_id = row['code']
    series_desc = row['description']
    
    for i in range(len(row['indicator'])):
    
        series_dict = {}
        
        goal_id =  row['goal'][i]
        target_id = row['target'][i]
        indicator_id = row['indicator'][i]
        release = row['release']
        
        series_dict['Goal'] = goal_id
        series_dict['Target'] = target_id
        series_dict['Indicator'] = indicator_id
        series_dict['Series'] = series_id
        series_dict['SeriesDesc'] = series_desc
        
        series_attr = pd.read_json('https://unstats.un.org/SDGAPI/v1/sdg/Series/'+series_id+'/Attributes')
        series_dict['Units'] = series_attr.loc[series_attr['id'] == 'Units'].iloc[0]['codes'][0]['sdmx']

        
        global_series.append(series_dict)

global_series = pd.DataFrame(global_series)
global_series.head(6)

## Global series catalogue

In [ ]:
global_series_catalogue = pd.merge(pd.merge(indicator_list,tier_classification, how='left', on=['IndicatorCode']),
                                   global_series[['Indicator', 'Series', 'SeriesDesc', 'Units']], how = 'left', on = ['Indicator'])
global_series_catalogue.head(5) 

In [ ]:
global_series_catalogue.shape

Add a marker indicating that this is from the global sdg indicators database:

In [ ]:
global_series_catalogue['side'] = 'Global'

In [ ]:
global_series_catalogue.to_excel(global_indicator_dir + 'global_series_catalogue.xlsx',index=False, encoding='UTF-8')